# GPU

In [1]:
import time
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms

In [2]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

DEVICE = torch.device(device)

In [3]:
DEVICE

device(type='cuda')

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [5]:
train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)


Extracting ./data/cifar-10-python.tar.gz to ./data


In [6]:
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

Files already downloaded and verified


## Sieć konwolucyjna

In [7]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(16 * 8 * 8, 128)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(128, 10)

    
    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.reshape(-1, 16 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [8]:
def training_loop(n_epochs, optimizer, model, criterion, train_loader):
    start = time.time()
    for epoch in range(n_epochs):
        losses = []
        
        for imgs, labels in train_loader:
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            
            losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        if epoch % 10 == 0:
            print(f"Epoch: {epoch}; loss: {sum(losses)/len(losses)}")
    end = time.time()
    print(f"Czas: {round(end - start)} s")

In [9]:
learning_rate = 0.001
n_epochs = 150

In [10]:
model = ConvNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [11]:
training_loop(n_epochs=n_epochs, optimizer=optimizer, model=model, 
              criterion=criterion, train_loader=train_loader)

Epoch: 0; loss: 2.2848520196612228
Epoch: 10; loss: 1.7692459887250915
Epoch: 20; loss: 1.5803487226176445
Epoch: 30; loss: 1.477790367115489
Epoch: 40; loss: 1.4035366534272118
Epoch: 50; loss: 1.332455146038319
Epoch: 60; loss: 1.267674306850604
Epoch: 70; loss: 1.2125169990770042
Epoch: 80; loss: 1.1644607158877966
Epoch: 90; loss: 1.1225203372480925
Epoch: 100; loss: 1.0856249242487466
Epoch: 110; loss: 1.0530244705774594
Epoch: 120; loss: 1.0246431932729834
Epoch: 130; loss: 0.9987449555293374
Epoch: 140; loss: 0.9739822141654656
Czas: 1337 s


In [12]:
def get_accuracy(model, data_loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for xs, ys in data_loader:
            xs = xs.to(DEVICE)
            ys = ys.to(DEVICE)
            scores = model(xs)
            _, predictions = torch.max(scores, dim=1)
            correct += (predictions == ys).sum()
            total += ys.shape[0]

        acc = float(correct) / float(total) * 100
        print(f"accuracy: {acc:.2f}")

In [13]:
get_accuracy(model, train_loader)

accuracy: 67.02


In [14]:
get_accuracy(model, test_loader)

accuracy: 64.46


## Zapis i wczytanie modelu

In [15]:
torch.save(model.state_dict(), "cifar10_model.pt")

In [16]:
loaded_model = ConvNet().to(DEVICE)
loaded_model.load_state_dict(torch.load("cifar10_model.pt"))

<All keys matched successfully>